# Neurosynth Compose Notebook

After you've built your meta-analysis on [neurosynth compose](compose.neurosynth.org), you can use this notebook to execute and view the meta-analytic results.

In [ ]:
#@title Run once to install NiMARE (~1 minute)
%%capture --no-display --no-stderr
%%bash
wget https://raw.githubusercontent.com/neurostuff/neurosynth-compose-notebook/main/requirements.txt
pip install -r requirements.txt

## INSERT YOUR META-ANALYSIS ID HERE

This is the id for your specific meta-analysis

In [ ]:
#@title Set your Neurosynth-Compose Meta-Analysis ID { run: "auto" }
META_ID = input("Paste the 12 character ID of the meta-analysis here: ")  # for example: "8dJ6tG7wfbyS"
META_ID = META_ID.replace(" ", "")

Paste the 12 character ID of the meta-analysis here:  6YA5FqTPNJEq


## Run your selected Meta-Analysis

A workflow in NiMARE will interpret the meta-analysis you specified and
run the selected algorithm and corrector (optional) on the studyset selected.

In [ ]:
from nimare.workflows import compose_workflow  #  special workflow to run meta-analysis

In [ ]:
if META_ID is not None:
    meta_result = compose_workflow(META_ID)
else:
    print("UPDATE META_ID VARIABLE TO YOUR META-ANALYSIS")

## Choose which statistical map to investigate

The Default is the z map, but each estimator/corrector combination may generate
corrected/more appropriate statistical maps for investigation.

In [ ]:
from nilearn.plotting import plot_stat_map  # display statistical images

In [ ]:
# here are your choices
print("\n".join(list(meta_result.maps.keys())))

In [ ]:
STAT_NAME = "z"
stat_map = meta_result.get_map(STAT_NAME)

In [ ]:
plot_stat_map(stat_map)

## Investigate what studies contributed to which clusters

Dive into your results to discover which analyses are driving your results.

In [ ]:
from nimare.diagnostics import FocusCounter # Only works for coordinate-based meta-analyses
fc = FocusCounter(target_image=STAT_NAME)

In [ ]:
contribution_table, cluster_img = fc.transform(meta_result)

In [ ]:
contribution_table